In [1]:
# Data processing
import numpy as np
import pandas as pd
import xarray as xr
import iris
import iris.pandas
import iris.coord_categorisation
import warnings
warnings.filterwarnings('ignore', module='iris')
import dateutil.parser
from datetime import datetime, timedelta
from pathlib import Path
from shapely.geometry import Point, Polygon
# Visualization
import cartopy.util
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from matplotlib.dates import DateFormatter
from matplotlib.ticker import FuncFormatter
# Plotting parameters
plt.rcParams['mathtext.default'] = 'regular'
def m2km(x, pos):
    '''Convert meters to kilometers when plotting axis labels'''
    return int(x*1e-3) #'{:1.1f}'.format(x*1e-3)
savefig = True
publish = True
if publish:
    mpl.rcParams['xtick.labelsize'] = 16
    mpl.rcParams['ytick.labelsize'] = 20
    mpl.rcParams['axes.titlesize'] = 22
    mpl.rcParams['axes.labelsize'] = 20
    plt.rcParams['font.size'] = 20

In [2]:
# varying range
species_names_dict = {'ch4': {'latex': '$CH_4$', 'abbr': 'RH_C1', 'ppn': 'ppbvC', 'max': 2100},
                      'c2h6': {'latex': '$C_2H_6$', 'abbr': 'RH_C2', 'ppn': 'pptvC', 'max': 6000}, 
                      'c3h8': {'latex': '$C_3H_8$', 'abbr': 'RH_C3', 'ppn': 'pptvC', 'max': 1000},
                      'o3': {'latex': '$O_3$', 'abbr': 'O3', 'ppn': 'ppbv', 'max': 600},
                      'oh': {'latex': 'OH', 'abbr': 'OH', 'ppn': 'pptv', 'max': 3.5},
                      'no': {'latex': 'NO', 'abbr': 'NO', 'ppn': 'ppbv', 'max': 0.2}, # 0.5
                      'meono2': {'latex': '$MeONO_2$', 'abbr': 'RONO2_C1', 'ppn': 'pptv', 'max': 70},
                      'etono2': {'latex': '$EtONO_2$', 'abbr': 'RONO2_C2', 'ppn': 'pptv', 'max': 10},
                      'nprono2': {'latex': '$nPrONO_2$', 'abbr': 'RONO2_C3n', 'ppn': 'pptv', 'max': 1},
                      'iprono2': {'latex': '$iPrONO_2$', 'abbr': 'RONO2_C3i', 'ppn': 'pptv', 'max': 10}}
# common range
# species_names_dict = {'ch4': {'latex': '$CH_4$', 'abbr': 'RH_C1', 'ppn': 'ppbvC', 'max': 2100},
#                       'c2h6': {'latex': '$C_2H_6$', 'abbr': 'RH_C2', 'ppn': 'pptvC', 'max': 8000}, 
#                       'c3h8': {'latex': '$C_3H_8$', 'abbr': 'RH_C3', 'ppn': 'pptvC', 'max': 8000},
#                       'meono2': {'latex': '$MeONO_2$', 'abbr': 'RONO2_C1', 'ppn': 'pptv', 'max': 70},
#                       'etono2': {'latex': '$EtONO_2$', 'abbr': 'RONO2_C2', 'ppn': 'pptv', 'max': 70},
#                       'nprono2': {'latex': '$nPrONO_2$', 'abbr': 'RONO2_C3n', 'ppn': 'pptv', 'max': 20},
#                       'iprono2': {'latex': '$iPrONO_2$', 'abbr': 'RONO2_C3i', 'ppn': 'pptv', 'max': 20}}

#### Read data

In [3]:
path_to_atom = Path('../../../../../../../obs/ATom/nc/data')
# Select flights
fname_atom_flight = 'MER-WAS_DC8_201702*.nc'
# Read ATom data
atom_dsinf = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), decode_times=True)
atom_dsmms = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='MMS', decode_cf=True)
atom_dswas = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='WAS', decode_cf=True)
atom_dsch4 = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='NOAA-Picarro', decode_cf=True)
atom_dsno = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='NOyO3-NO', decode_cf=True)
atom_dso3 = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='NOyO3-O3', decode_cf=True)
atom_dsoh = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='ATHOS-HOx', decode_cf=True)

In [4]:
# Choose UKCA run
ukca_run_name = 'xojnh'
compared_common_id = '170201_170221_1702_merge'
path_to_ukca = Path('../../../../../processed') / ukca_run_name
# Read UKCA data
cb_nyrs_ch4 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rh.nc'), 'ch4')
cb_nyrs_c2h6 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rh.nc'), 'c2h6')
cb_nyrs_c3h8 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rh.nc'), 'c3h8')
cb_nyrs_o3 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_ox.nc'), 'o3')
cb_nyrs_oh = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_ox.nc'), 'oh')
cb_nyrs_no = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_nox.nc'), 'no')
cb_nyrs_meono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rono2.nc'), 'meono2')
cb_nyrs_etono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rono2.nc'), 'etono2')
cb_nyrs_nprono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rono2.nc'), 'nprono2')
if ukca_run_name == 'xojng' or ukca_run_name == 'xojnc':
    cb_nyrs_iprono2 = cb_nyrs_nprono2 # no iPrONO2 variable in BASE or old BB run, while nPrONO2 was zero
else:
    cb_nyrs_iprono2 = iris.load_cube(str(path_to_ukca / f'{ukca_run_name}_relvl_rono2.nc'), 'iprono2')
# Load UKCA geospatial coordinates
ukca_lats = iris.load_cube(str(Path('../../../../../data') / 'um_orography_xnvtj.nc'), 'OROGRAPHY (/STRAT LOWER BC)').coord('latitude')
ukca_lons = iris.load_cube(str(Path('../../../../../data') / 'um_orography_xnvtj.nc'), 'OROGRAPHY (/STRAT LOWER BC)').coord('longitude')

In [5]:
# Add month and year coordinates
iris.coord_categorisation.add_month(cb_nyrs_ch4, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_c2h6, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_c3h8, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_o3, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_oh, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_no, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_meono2, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_etono2, 'time', name='month')
iris.coord_categorisation.add_month(cb_nyrs_nprono2, 'time', name='month')
if not ukca_run_name == 'xojng':
    iris.coord_categorisation.add_month(cb_nyrs_iprono2, 'time', name='month')
# Calculate monthly mean from 8 years of data
cb_ch4 = cb_nyrs_ch4[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_c2h6 = cb_nyrs_c2h6[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_c3h8 = cb_nyrs_c3h8[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_o3 = cb_nyrs_o3[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_oh = cb_nyrs_oh[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_no = cb_nyrs_no[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_meono2 = cb_nyrs_meono2[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_etono2 = cb_nyrs_etono2[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_nprono2 = cb_nyrs_nprono2[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)
cb_iprono2 = cb_nyrs_iprono2[24:120].extract(iris.Constraint(month='Feb')).aggregated_by(['month'], iris.analysis.MEAN)

In [6]:
# Path to figures
path_to_figs = Path('../../../../../../../results') / ukca_run_name / compared_common_id
path_to_figs.mkdir(exist_ok=True) # create a new folder if it doesn't exist

#### Select geographical region/s for spacial averaging

In [7]:
# Extract datetimes from ATom data
atom_datetime = atom_dsinf.time.values.astype('<M8[us]').astype(datetime)
atom_date_strt = atom_datetime[0]
atom_date_stop = atom_datetime[-1]
# Extract spatial coordinates from ATom data
sample_lats = np.asarray(atom_dsmms.G_LAT.data) 
sample_lons = np.asarray(atom_dsmms.G_LONG.data)
sample_alts = np.asarray(atom_dsmms.G_ALT.data)
# Create UKCA lat lon grid
grid_lons, grid_lats = np.meshgrid(ukca_lons.points, ukca_lats.points)
# Construct pairs of coordinate points
sample_lon_lat_points = []
for i, j in zip(sample_lons, sample_lats):
    sample_lon_lat_points.append(Point(i, j))

In [8]:
# Define a geographical region
r1_llon_ukca, r1_ulon_ukca = 195.9375, 214.6875
r1_llon, r1_ulon = r1_llon_ukca-360, r1_ulon_ukca-360
r1_llat, r1_ulat = 21.875, 53.125
r1 = Polygon([(r1_llon, r1_llat), (r1_llon, r1_ulat), (r1_ulon, r1_ulat), (r1_ulon, r1_llat)])
r2_llon_ukca, r2_ulon_ukca = 169.6875, 201.5625
r2_1_llon, r2_1_ulon = r2_llon_ukca, 180
r2_2_llon, r2_2_ulon = -180, r2_ulon_ukca-360
r2_llat, r2_ulat = -40.625, 20.625
r2_1 = Polygon([(r2_1_llon, r2_llat), (r2_1_llon, r2_ulat), (r2_1_ulon, r2_ulat), (r2_1_ulon, r2_llat)])
r2_2 = Polygon([(r2_2_llon, r2_llat), (r2_2_llon, r2_ulat), (r2_2_ulon, r2_ulat), (r2_2_ulon, r2_llat)])
r3_llon_ukca, r3_ulon_ukca = 180.9375, 285.9375
r3_llon, r3_ulon = r3_llon_ukca-360, r3_ulon_ukca-360
r3_llat, r3_ulat = -70.625, -50.625
r3 = Polygon([(r3_llon, r3_llat), (r3_llon, r3_ulat), (r3_ulon, r3_ulat), (r3_ulon, r3_llat)])
r4_llon_ukca, r4_ulon_ukca = 312.1875, 334.6875
r4_llon, r4_ulon = r4_llon_ukca-360, r4_ulon_ukca-360
r4_llat, r4_ulat = -45.625, -25.625
r4 = Polygon([(r4_llon, r4_llat), (r4_llon, r4_ulat), (r4_ulon, r4_ulat), (r4_ulon, r4_llat)])
r5_llon_ukca, r5_ulon_ukca = 323.4375, 344.0625
r5_llon, r5_ulon = r5_llon_ukca-360, r5_ulon_ukca-360
r5_llat, r5_ulat = -6.875, 36.875
r5 = Polygon([(r5_llon, r5_llat), (r5_llon, r5_ulat), (r5_ulon, r5_ulat), (r5_ulon, r5_llat)])
r6_llon_ukca, r6_ulon_ukca = 321.5625, 338.4375
r6_llon, r6_ulon = r6_llon_ukca-360, r6_ulon_ukca-360
r6_llat, r6_ulat = 38.125, 63.125
r6 = Polygon([(r6_llon, r6_llat), (r6_llon, r6_ulat), (r6_ulon, r6_ulat), (r6_ulon, r6_llat)])
r7_llon_ukca, r7_ulon_ukca = 269.0625, 321.5625
r7_llon, r7_ulon = r7_llon_ukca-360, r7_ulon_ukca-360
r7_llat, r7_ulat = 59.375, 81.875
r7 = Polygon([(r7_llon, r7_llat), (r7_llon, r7_ulat), (r7_ulon, r7_ulat), (r7_ulon, r7_llat)])
# r8_llon_ukca, r8_ulon_ukca = 237.1875, 269.0625
# r8_llon, r8_ulon = r8_llon_ukca-360, r8_ulon_ukca-360
# r8_llat, r8_ulat = 31.875, 50.625
# r8 = Polygon([(r8_llon, r8_llat), (r8_llon, r8_ulat), (r8_ulon, r8_ulat), (r8_ulon, r8_llat)])
r9_llon_ukca, r9_ulon_ukca = 201.5625, 233.4375 # between 234 and 237.1875 there is a spike in alkanes!
r9_llon, r9_ulon = r9_llon_ukca-360, r9_ulon_ukca-360
r9_llat, r9_ulat = 61.875, 81.875
r9 = Polygon([(r9_llon, r9_llat), (r9_llon, r9_ulat), (r9_ulon, r9_ulat), (r9_ulon, r9_llat)])
# Find points within a region
r1_points_within = []
r2_1_points_within, r2_2_points_within= [], []
r3_points_within = []
r4_points_within = []
r5_points_within = []
r6_points_within = []
r7_points_within = []
# r8_points_within = []
r9_points_within = []
for p in sample_lon_lat_points:
    r1_points_within.append(p.within(r1))
    r2_1_points_within.append(p.within(r2_1))
    r2_2_points_within.append(p.within(r2_2))
    r3_points_within.append(p.within(r3))
    r4_points_within.append(p.within(r4))
    r5_points_within.append(p.within(r5))
    r6_points_within.append(p.within(r6))
    r7_points_within.append(p.within(r7))
#     r8_points_within.append(p.within(r8))
    r9_points_within.append(p.within(r9))
r2_points_within = list(np.asarray(r2_1_points_within) | np.asarray(r2_2_points_within))
# Combine points within and region corners into lists
points_within_regions = [r1_points_within, r2_points_within, r3_points_within, r4_points_within, r5_points_within, 
                         r6_points_within, r7_points_within, r9_points_within]
ukca_regions_corners = [[r1_llon_ukca, r1_ulon_ukca, r1_llat, r1_ulat], [r2_llon_ukca, r2_ulon_ukca, r2_llat, r2_ulat], 
                        [r3_llon_ukca, r3_ulon_ukca, r3_llat, r3_ulat], [r4_llon_ukca, r4_ulon_ukca, r4_llat, r4_ulat], 
                        [r5_llon_ukca, r5_ulon_ukca, r5_llat, r5_ulat], [r6_llon_ukca, r6_ulon_ukca, r6_llat, r6_ulat], 
                        [r7_llon_ukca, r7_ulon_ukca, r7_llat, r7_ulat], [r9_llon_ukca, r9_ulon_ukca, r9_llat, r9_ulat]]
region_names = ['N Pacific', 'Central Pacific', 'S Pacific', 'SE Atlantic', 'Central Atlantic', 'N Atlantic', 'Greenland', 'W Canada']
# points_within_regions = [r1_points_within, r2_points_within, r3_points_within, r4_points_within, r5_points_within, 
#                          r6_points_within, r7_points_within, r8_points_within, r9_points_within]
# ukca_regions_corners = [[r1_llon_ukca, r1_ulon_ukca, r1_llat, r1_ulat], [r2_llon_ukca, r2_ulon_ukca, r2_llat, r2_ulat], 
#                         [r3_llon_ukca, r3_ulon_ukca, r3_llat, r3_ulat], [r4_llon_ukca, r4_ulon_ukca, r4_llat, r4_ulat], 
#                         [r5_llon_ukca, r5_ulon_ukca, r5_llat, r5_ulat], [r6_llon_ukca, r6_ulon_ukca, r6_llat, r6_ulat], 
#                         [r7_llon_ukca, r7_ulon_ukca, r7_llat, r7_ulat], [r8_llon_ukca, r8_ulon_ukca, r8_llat, r8_ulat], 
#                         [r9_llon_ukca, r9_ulon_ukca, r9_llat, r9_ulat]]
# region_names = ['N Pacific', 'Central Pacific', 'S Pacific', 'SE Atlantic', 'Central Atlantic', 'N Atlantic', 'Greenland', 'Central and W USA', 'W Canada']

In [9]:
def plot_region(lons, lats, points_within, ukca_llon, ukca_ulon, ukca_llat, ukca_ulat, llon, ulon, llat, ulat):
    if (abs(llon) <= 180 <= abs(ulon)) or (abs(llon) >= 180 >= abs(ulon)):
        projection = ccrs.PlateCarree(central_longitude=180)
    else:
        projection = ccrs.PlateCarree()
    fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(projection=projection), facecolor='w')
    ax.scatter(lons, lats, transform=ccrs.PlateCarree())
    ax.scatter(lons[points_within], lats[points_within], color='r', transform=ccrs.PlateCarree())
    ax.add_patch(mpatches.Rectangle(xy=[ukca_llon, ukca_llat], width=ukca_ulon-ukca_llon, height=ukca_ulat-ukca_llat, facecolor='b', alpha=0.2, transform=ccrs.PlateCarree()))
#     ax.plot(grid_lons, grid_lats, color='b', alpha=0.7, transform=ccrs.PlateCarree())
#     ax.plot(grid_lons.T, grid_lats.T, color='b', alpha=0.7, transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent([llon, ulon, llat, ulat], crs=ccrs.PlateCarree())
    ax.gridlines(draw_labels=False)

In [10]:
# plot_region(sample_lons, sample_lats, r1_points_within, r1_llon_ukca, r1_ulon_ukca, r1_llat, r1_ulat, -170, -140, 15, 60)

In [11]:
# plot_region(sample_lons, sample_lats, r2_points_within, r2_llon_ukca, r2_ulon_ukca, r2_llat, r2_ulat, -200, -120, -50, 40)

In [12]:
# plot_region(sample_lons, sample_lats, r3_points_within, r3_llon_ukca, r3_ulon_ukca, r3_llat, r3_ulat, -190, -60, -90, 0)

In [13]:
# plot_region(sample_lons, sample_lats, r4_points_within, r4_llon_ukca, r4_ulon_ukca, r4_llat, r4_ulat, -80, 20, -60, 0)

In [14]:
# plot_region(sample_lons, sample_lats, r5_points_within, r5_llon_ukca, r5_ulon_ukca, r5_llat, r5_ulat, -80, 20, -60, 60)

In [15]:
# plot_region(sample_lons, sample_lats, r6_points_within, r6_llon_ukca, r6_ulon_ukca, r6_llat, r6_ulat, -80, 20, 0, 80)

In [16]:
# plot_region(sample_lons, sample_lats, r7_points_within, r7_llon_ukca, r7_ulon_ukca, r7_llat, r7_ulat, -100, 0, 0, 90)

In [17]:
# plot_region(sample_lons, sample_lats, r8_points_within, r8_llon_ukca, r8_ulon_ukca, r8_llat, r8_ulat, -140, -80, 20, 90)

In [18]:
# plot_region(sample_lons, sample_lats, r9_points_within, r9_llon_ukca, r9_ulon_ukca, r9_llat, r9_ulat, -170, -80, 20, 90)

In [19]:
def plot_regions(lons, lats, points_within_regions, ukca_regions_corners, llon, ulon, llat, ulat):
    r1_points_within, r2_points_within, r3_points_within, r4_points_within, r5_points_within, r6_points_within, r7_points_within, r9_points_within = points_within_regions
    [r1_llon, r1_ulon, r1_llat, r1_ulat] = ukca_regions_corners[0]
    [r2_llon, r2_ulon, r2_llat, r2_ulat] = ukca_regions_corners[1]
    [r3_llon, r3_ulon, r3_llat, r3_ulat] = ukca_regions_corners[2]
    [r4_llon, r4_ulon, r4_llat, r4_ulat] = ukca_regions_corners[3]
    [r5_llon, r5_ulon, r5_llat, r5_ulat] = ukca_regions_corners[4]
    [r6_llon, r6_ulon, r6_llat, r6_ulat] = ukca_regions_corners[5]
    [r7_llon, r7_ulon, r7_llat, r7_ulat] = ukca_regions_corners[6]
#     [r8_llon, r8_ulon, r8_llat, r8_ulat] = ukca_regions_corners[7]
    [r9_llon, r9_ulon, r9_llat, r9_ulat] = ukca_regions_corners[8]
    
    fig, ax = plt.subplots(figsize=(10,7), subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)), facecolor='w')
    ax.scatter(lons, lats, color='grey', transform=ccrs.PlateCarree())
    kwargs_sc = dict(color='k', transform=ccrs.PlateCarree())
    ax.scatter(lons[r1_points_within], lats[r1_points_within], **kwargs_sc)
    ax.scatter(lons[r2_points_within], lats[r2_points_within], **kwargs_sc)
    ax.scatter(lons[r3_points_within], lats[r3_points_within], **kwargs_sc)
    ax.scatter(lons[r4_points_within], lats[r4_points_within], **kwargs_sc)
    ax.scatter(lons[r5_points_within], lats[r5_points_within], **kwargs_sc)
    ax.scatter(lons[r6_points_within], lats[r6_points_within], **kwargs_sc)
    ax.scatter(lons[r7_points_within], lats[r7_points_within], **kwargs_sc)
#     ax.scatter(lons[r8_points_within], lats[r8_points_within], **kwargs_sc)
    ax.scatter(lons[r9_points_within], lats[r9_points_within], **kwargs_sc)
    kwargs_patch = dict(facecolor='r', alpha=0.2, edgecolor='k', transform=ccrs.PlateCarree())
    ax.add_patch(mpatches.Rectangle(xy=[r1_llon, r1_llat], width=r1_ulon-r1_llon, height=r1_ulat-r1_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r2_llon, r2_llat], width=r2_ulon-r2_llon, height=r2_ulat-r2_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r3_llon, r3_llat], width=r3_ulon-r3_llon, height=r3_ulat-r3_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r4_llon, r4_llat], width=r4_ulon-r4_llon, height=r4_ulat-r4_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r5_llon, r5_llat], width=r5_ulon-r5_llon, height=r5_ulat-r5_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r6_llon, r6_llat], width=r6_ulon-r6_llon, height=r6_ulat-r6_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r7_llon, r7_llat], width=r7_ulon-r7_llon, height=r7_ulat-r7_llat, **kwargs_patch))
#     ax.add_patch(mpatches.Rectangle(xy=[r8_llon, r8_llat], width=r8_ulon-r8_llon, height=r8_ulat-r8_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r9_llon, r9_llat], width=r9_ulon-r9_llon, height=r9_ulat-r9_llat, **kwargs_patch))
    ax.set_title('ATom', loc='right')
    ax.set_title(f'{atom_date_strt.strftime("%b %Y")}', loc='left')
    ax.set_extent([llon, ulon, llat, ulat], crs=ccrs.PlateCarree())
    ax.coastlines()
    if publish:
        fig.savefig(path_to_figs / 'publish' / f'{ukca_run_name}_{compared_common_id}_regions.png', bbox_inches='tight')

In [20]:
# plot_regions(sample_lons, sample_lats, points_within_regions, ukca_regions_corners, -200, -5, -90, 90)

#### Process ATom data

In [21]:
# Convert ATom C2H6, C2H4, C2H2, C3H8, C3H6 to pptvC to match UKCA lumped species, namely C2H6=C2H6+C2H4+C2H2, C3H8=C3H8+C3H6
atom_dswas_c2h6 = atom_dswas['Ethane_WAS']*2 + atom_dswas['Ethene_WAS']*2 + atom_dswas['Ethyne_WAS']*2
atom_dswas_c3h8 = atom_dswas['Propane_WAS']*3 + atom_dswas['Propene_WAS']*3
# Convert xarray dataset with observational data to pandas dataframe
atom_ch4 = atom_dsch4.CH4_NOAA.to_dataframe(name='ch4')
atom_c2h6 = atom_dswas_c2h6.to_dataframe(name='c2h6')
atom_c3h8 = atom_dswas_c3h8.to_dataframe(name='c3h8')
atom_o3 = atom_dso3['O3_CL'].to_dataframe(name='o3')
atom_oh = atom_dsoh['OH_pptv'].to_dataframe(name='oh')
atom_no = atom_dsno['NO_CL'].to_dataframe(name='no')
atom_meono2 = atom_dswas['MeONO2_WAS'].to_dataframe(name='meono2')
atom_etono2 = atom_dswas['EtONO2_WAS'].to_dataframe(name='etono2')
atom_nprono2 = atom_dswas['n-PrONO2_WAS'].to_dataframe(name='nprono2')
atom_iprono2 = atom_dswas['i-PrONO2_WAS'].to_dataframe(name='iprono2')

In [22]:
# Select longitudes and latitudes where ATom data is available
lons_ch4_notnan = np.where(atom_ch4.ch4.isna()==False, sample_lons, atom_ch4.ch4)
lats_ch4_notnan = np.where(atom_ch4.ch4.isna()==False, sample_lats, atom_ch4.ch4)
lons_c2h6_notnan = np.where(atom_c2h6.c2h6.isna()==False, sample_lons, atom_c2h6.c2h6)
lats_c2h6_notnan = np.where(atom_c2h6.c2h6.isna()==False, sample_lats, atom_c2h6.c2h6)
lons_c3h8_notnan = np.where(atom_c3h8.c3h8.isna()==False, sample_lons, atom_c3h8.c3h8)
lats_c3h8_notnan = np.where(atom_c3h8.c3h8.isna()==False, sample_lats, atom_c3h8.c3h8)
lons_o3_notnan = np.where(atom_o3.o3.isna()==False, sample_lons, atom_o3.o3)
lats_o3_notnan = np.where(atom_o3.o3.isna()==False, sample_lats, atom_o3.o3)
lons_oh_notnan = np.where(atom_oh.oh.isna()==False, sample_lons, atom_oh.oh)
lats_oh_notnan = np.where(atom_oh.oh.isna()==False, sample_lats, atom_oh.oh)
lons_no_notnan = np.where(atom_no.no.isna()==False, sample_lons, atom_no.no)
lats_no_notnan = np.where(atom_no.no.isna()==False, sample_lats, atom_no.no)
lons_meono2_notnan = np.where(atom_meono2.meono2.isna()==False, sample_lons, atom_meono2.meono2)
lats_meono2_notnan = np.where(atom_meono2.meono2.isna()==False, sample_lats, atom_meono2.meono2)
lons_etono2_notnan = np.where(atom_etono2.etono2.isna()==False, sample_lons, atom_etono2.etono2)
lats_etono2_notnan = np.where(atom_etono2.etono2.isna()==False, sample_lats, atom_etono2.etono2)
lons_nprono2_notnan = np.where(atom_nprono2.nprono2.isna()==False, sample_lons, atom_nprono2.nprono2)
lats_nprono2_notnan = np.where(atom_nprono2.nprono2.isna()==False, sample_lats, atom_nprono2.nprono2)
lons_iprono2_notnan = np.where(atom_iprono2.iprono2.isna()==False, sample_lons, atom_iprono2.iprono2)
lats_iprono2_notnan = np.where(atom_iprono2.iprono2.isna()==False, sample_lats, atom_iprono2.iprono2)
lons_notnan = [lons_ch4_notnan, lons_c2h6_notnan, lons_c3h8_notnan, lons_o3_notnan, lons_oh_notnan, lons_no_notnan,
               lons_meono2_notnan, lons_etono2_notnan, lons_nprono2_notnan, lons_iprono2_notnan]
lats_notnan = [lats_ch4_notnan, lats_c2h6_notnan, lats_c3h8_notnan, lats_o3_notnan, lats_oh_notnan, lats_no_notnan,
               lats_meono2_notnan, lats_etono2_notnan, lats_nprono2_notnan, lats_iprono2_notnan]

In [23]:
def plot_regions_by_specie(lons, lats, points_within_regions, ukca_regions_corners, llon, ulon, llat, ulat, specie_name, abbr):
    r1_points_within, r2_points_within, r3_points_within, r4_points_within, r5_points_within, r6_points_within, r7_points_within, r9_points_within = points_within_regions
    [r1_llon, r1_ulon, r1_llat, r1_ulat] = ukca_regions_corners[0]
    [r2_llon, r2_ulon, r2_llat, r2_ulat] = ukca_regions_corners[1]
    [r3_llon, r3_ulon, r3_llat, r3_ulat] = ukca_regions_corners[2]
    [r4_llon, r4_ulon, r4_llat, r4_ulat] = ukca_regions_corners[3]
    [r5_llon, r5_ulon, r5_llat, r5_ulat] = ukca_regions_corners[4]
    [r6_llon, r6_ulon, r6_llat, r6_ulat] = ukca_regions_corners[5]
    [r7_llon, r7_ulon, r7_llat, r7_ulat] = ukca_regions_corners[6]
#     [r8_llon, r8_ulon, r8_llat, r8_ulat] = ukca_regions_corners[7]
    [r9_llon, r9_ulon, r9_llat, r9_ulat] = ukca_regions_corners[8]
    
    fig, ax = plt.subplots(figsize=(7,7), subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)), facecolor='w')
    kwargs_sc = dict(color='k', transform=ccrs.PlateCarree())
    ax.scatter(lons[r1_points_within], lats[r1_points_within], **kwargs_sc)
    ax.scatter(lons[r2_points_within], lats[r2_points_within], **kwargs_sc)
    ax.scatter(lons[r3_points_within], lats[r3_points_within], **kwargs_sc)
    ax.scatter(lons[r4_points_within], lats[r4_points_within], **kwargs_sc)
    ax.scatter(lons[r5_points_within], lats[r5_points_within], **kwargs_sc)
    ax.scatter(lons[r6_points_within], lats[r6_points_within], **kwargs_sc)
    ax.scatter(lons[r7_points_within], lats[r7_points_within], **kwargs_sc)
#     ax.scatter(lons[r8_points_within], lats[r8_points_within], **kwargs_sc)
    ax.scatter(lons[r9_points_within], lats[r9_points_within], **kwargs_sc)
    kwargs_patch = dict(facecolor='red', alpha=0.2, edgecolor='k', transform=ccrs.PlateCarree())
    ax.add_patch(mpatches.Rectangle(xy=[r1_llon, r1_llat], width=r1_ulon-r1_llon, height=r1_ulat-r1_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r2_llon, r2_llat], width=r2_ulon-r2_llon, height=r2_ulat-r2_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r3_llon, r3_llat], width=r3_ulon-r3_llon, height=r3_ulat-r3_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r4_llon, r4_llat], width=r4_ulon-r4_llon, height=r4_ulat-r4_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r5_llon, r5_llat], width=r5_ulon-r5_llon, height=r5_ulat-r5_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r6_llon, r6_llat], width=r6_ulon-r6_llon, height=r6_ulat-r6_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r7_llon, r7_llat], width=r7_ulon-r7_llon, height=r7_ulat-r7_llat, **kwargs_patch))
#     ax.add_patch(mpatches.Rectangle(xy=[r8_llon, r8_llat], width=r8_ulon-r8_llon, height=r8_ulat-r8_llat, **kwargs_patch))
    ax.add_patch(mpatches.Rectangle(xy=[r9_llon, r9_llat], width=r9_ulon-r9_llon, height=r9_ulat-r9_llat, **kwargs_patch))
    ax.set_title('ATom', loc='center')
    ax.set_title(f'{specie_name}', loc='right')
    ax.set_title(f'{atom_date_strt.strftime("%b %Y")}', loc='left')
    ax.set_extent([llon, ulon, llat, ulat], crs=ccrs.PlateCarree())
    ax.coastlines()
    if publish:
        fig.savefig(path_to_figs / 'publish' / f'{ukca_run_name}_{compared_common_id}_regions_{abbr}.png', bbox_inches='tight')

In [24]:
# for (specie, specie_dict), lons, lats in zip(species_names_dict.items(), lons_notnan, lats_notnan):
#     plot_regions_by_specie(lons, lats, points_within_regions, ukca_regions_corners, -200, -5, -90, 90, specie_dict['latex'], specie_dict['abbr'])

In [25]:
# def plot_ts(atom, alts):
#     fig, ax = plt.subplots()
#     ax.plot(atom, marker='o')
#     ax2 = ax.twinx()
#     ax2.plot(alts, marker='o', color='grey', linestyle='--')

In [26]:
# plot_ts(atom_meono2[r1_points_within].values, atom_dsmms.G_ALT[r1_points_within].data)

In [27]:
# plot_ts(atom_meono2[r2_points_within].values, atom_dsmms.G_ALT[r2_points_within].data)

In [28]:
# plot_ts(atom_meono2[r3_points_within].values, atom_dsmms.G_ALT[r3_points_within].data)

In [29]:
# Choose bins
alt_bins = np.arange(0, 14000, 500)
alt_bin_inds = np.arange(0, len(alt_bins)-1, 1)
alt_bin_mids = np.arange(250, 13500, 500)
alt_r1_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r1_points_within], alt_bins)
alt_r2_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r2_points_within], alt_bins)
alt_r3_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r3_points_within], alt_bins)
alt_r4_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r4_points_within], alt_bins)
alt_r5_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r5_points_within], alt_bins)
alt_r6_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r6_points_within], alt_bins)
alt_r7_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r7_points_within], alt_bins)
# alt_r8_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r8_points_within], alt_bins)
alt_r9_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r9_points_within], alt_bins)
alt_regions_bin_inds = [alt_r1_bin_inds, alt_r2_bin_inds, alt_r3_bin_inds,
                        alt_r4_bin_inds, alt_r5_bin_inds, alt_r6_bin_inds,
                        alt_r7_bin_inds, alt_r9_bin_inds]
# alt_regions_bin_inds = [alt_r1_bin_inds, alt_r2_bin_inds, alt_r3_bin_inds,
#                         alt_r4_bin_inds, alt_r5_bin_inds, alt_r6_bin_inds,
#                         alt_r7_bin_inds, alt_r8_bin_inds, alt_r9_bin_inds]
# Bin observational data
atom2process = [atom_ch4, atom_c2h6, atom_c3h8, atom_o3, atom_oh, atom_no, atom_meono2, atom_etono2, atom_nprono2, atom_iprono2]
atom_regional_vps = []
for points_within_region, alt_region_bin_inds in zip(points_within_regions, alt_regions_bin_inds):
    specie_dict = {}
    for specie in atom2process:
        stat_dict = {}
        stat_dict['mean'] = specie[points_within_region].groupby(alt_region_bin_inds).mean().reindex(index=alt_bin_inds, fill_value=np.nan)
        stat_dict['std'] = specie[points_within_region].groupby(alt_region_bin_inds).std().reindex(index=alt_bin_inds, fill_value=np.nan)
        specie_dict[specie.columns[0]] = stat_dict
    atom_regional_vps.append(specie_dict)

#### Process UKCA data

In [30]:
# Add coordinate bounds (for calculating area weights)
ukca2process = iris.cube.CubeList([cb_ch4, cb_c2h6, cb_c3h8, cb_o3, cb_oh, cb_no, cb_meono2, cb_etono2, cb_nprono2, cb_iprono2])
for cube in ukca2process:
    for coord in ['longitude', 'latitude']:
            if not cube.coord(coord).has_bounds():
                cube.coord(coord).guess_bounds()

In [31]:
# Calculate area weights for each region
any_cube = cb_meono2
regions = []
for corners in ukca_regions_corners:
    llon, ulon, llat, ulat = corners
    lonlat_constraint = iris.Constraint(longitude=lambda cell: llon <= cell <= ulon, 
                                        latitude=lambda cell: llat <= cell <= ulat)
    any_cube_extr = any_cube.extract(lonlat_constraint)
    weights_cube = any_cube_extr.copy(data=iris.analysis.cartography.area_weights(any_cube_extr))
    weights_cube.rename('area_weights')
    regions.append(
         {'corners': {k: v for k, v in zip(['llon', 'ulon', 'llat', 'ulat'], corners)},
          'area_weights': weights_cube})

In [32]:
# Calculate regional mean vertical profiles and standart deviations
stats = ('mean', 'std_dev')
ukca_regional_vps = []
for region in regions:
    cube_dict = {}
    corners = region['corners']
    lonlat_constraint = iris.Constraint(longitude=lambda cell: corners['llon'] <= cell <= corners['ulon'], 
                                        latitude=lambda cell: corners['llat'] <= cell <= corners['ulat'])    
    for cube in ukca2process.extract(lonlat_constraint):
        stat_dict = {}
        for stat in stats:
            cubelist = iris.cube.CubeList()
            for lbound, ubound in zip(alt_bins[:-1], alt_bins[1:]):
                alt_constraint = iris.Constraint(altitude=lambda cell: lbound < cell <= ubound)              
                cube_extr = cube.extract(alt_constraint)
                if stat == 'mean':
                    kwargs = {'weights': region['area_weights'].extract(alt_constraint).data}
                else:
                    kwargs = {}
                cubelist.append(cube_extr.collapsed(['longitude', 'latitude', 'altitude'], getattr(iris.analysis, stat.upper()), **kwargs))
            stat_dict[stat] = cubelist.merge_cube()
        cube_dict[cube.name()] = stat_dict
    ukca_regional_vps.append(cube_dict)

In [33]:
# Store vertical profiles for plotting in another notebook
ukca_regional_vps_xojnh = ukca_regional_vps
%store ukca_regional_vps_xojnh

Stored 'ukca_regional_vps_xojnh' (list)


In [34]:
break

SyntaxError: 'break' outside loop (cell_name, line 4)

In [ ]:
a = []
b = []
sp = 'meono2'
for i, r in enumerate(region_names):
    a.append(np.max((ukca_regional_vps[i][sp]['mean'].data + ukca_regional_vps[i][sp]['std_dev'].data)))
    b.append(np.nanmax((atom_regional_vps[i][sp]['mean'].values + atom_regional_vps[i][sp]['std'].values)))
print(np.max(a), np.nanmax(b))

#### Plot region average vertical profile

In [ ]:
def plot_vp(atom_mean, ukca_mean, atom_std, ukca_std, region_name, region_number, specie_name, specie_abbr, specie_ppn, specie_max):
    '''Plot area averaged vertical profile.'''
    fig, ax = plt.subplots(figsize=(7,7), facecolor='w')
    ax.errorbar(atom_mean, alt_bin_mids, xerr=atom_std, color='k', fmt='--o', linewidth=1, elinewidth=0.8, capsize=2)
    ax.errorbar(ukca_mean, alt_bin_mids, xerr=ukca_std, color='r', fmt='--o', linewidth=1, elinewidth=0.8, capsize=2)
    # common
    ax.set_title(f'{specie_name}', loc='right')
    ax.set_title(f'{region_name}', loc='center')
    ax.set_title(f'{atom_date_strt.strftime("%b")}', loc='left')
    ax.set_xlabel(f'{specie_ppn}')
    ax.set_xlim(0, specie_max)
    try:
        ax.set_xlim(0, max(np.nanmax(atom_mean+atom_std), np.nanmax(ukca_mean+ukca_std)))
    except ValueError:
        pass
    ax.set_ylabel('Altitude, km')
    ax.set_ylim(-200,13500)
    ax.set_yticks(alt_bins)
    ax.set_yticklabels(map(int,(alt_bins/1000)))
    ax.legend(['ATom', f'UKCA'], frameon=False)
    for ytcklbl in ax.yaxis.get_ticklabels()[1::2]:
        ytcklbl.set_visible(False)
    if publish:
        fig.savefig(path_to_figs / 'publish' / f'{ukca_run_name}_{compared_common_id}_vp_r{region_number}_{specie_abbr}.png', bbox_inches='tight')

In [ ]:
# for specie, specie_dict in species_names_dict.items():
#     for n, region_name in enumerate(region_names):
#         plot_vp(atom_regional_vps[n][specie]['mean'].values, 
#                 ukca_regional_vps[n][specie]['mean'].data, 
#                 atom_regional_vps[n][specie]['std'].values, 
#                 ukca_regional_vps[n][specie]['std_dev'].data, region_name, n+1, specie_dict['latex'], specie_dict['abbr'], specie_dict['ppn'], specie_dict['max'])